In [3]:
# Import required packages
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output



In [4]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [19]:
launch_sites = spacex_df['Launch Site'].unique().tolist()
        # Get min and max payload values from the dataframe
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

In [22]:
app = Dash(__name__)

# Create dropdown options list
dropdown_options = [{'label': 'All Sites', 'value': 'ALL'}] + [
    {'label': site, 'value': site} for site in launch_sites
]

# App layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36',
                                                      'font-size': 40}),
    
    # Dropdown for Launch Sites
    dcc.Dropdown(
        id='site-dropdown',
        options=dropdown_options,
        value='ALL',  # Default dropdown value
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),

    # Pie chart placeholder
    html.Div(dcc.Graph(id='success-pie-chart')),

    html.Br(),

    # Range Slider
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
        value=[min_payload, max_payload]
    ),

    html.Br(),

    # Scatter plot placeholder
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# Callback to update pie chart based on dropdown selection
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        # Generate a pie chart for total success launches by site
        fig = px.pie(
            spacex_df,
            values='class',  # Sum of success (class=1) counts
            names='Launch Site',
            title='Total Success Launches by Site'
        )
        return fig
    else:
        # Filter the dataframe for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        # Count successes (class=1) and failures (class=0)
        site_counts = filtered_df['class'].value_counts().reset_index()
        site_counts.columns = ['class', 'count']
        # Create pie chart for the selected site
        fig = px.pie(
            site_counts,
            values='count',
            names='class',
            title=f'Total Success vs Failure Launches for site {entered_site}'
        )
        return fig

# Callback for updating scatter plot based on dropdown and range slider
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_plot(selected_site, payload_range):
    # Filter data based on payload slider range
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)]
    
    if selected_site == 'ALL':
        # Scatter plot for all sites
        fig = px.scatter(
            filtered_df,
            x='Payload Mass (kg)', y='class',
            color='Booster Version Category',
            title='Correlation between Payload and Success for all Sites'
        )
    else:
        # Filter data for the selected site
        site_df = filtered_df[filtered_df['Launch Site'] == selected_site]
        fig = px.scatter(
            site_df,
            x='Payload Mass (kg)', y='class',
            color='Booster Version Category',
            title=f'Correlation between Payload and Success for site {selected_site}'
        )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)